In [ ]:
import pandas as pd
from datetime import date
from dataclasses import dataclass


@dataclass
class Candle:
    timestamp: object
    open: float
    high: float
    low: float
    close: float


class LiveYesterdayHighLowTrader:
    def __init__(
        self,
        symbol: str,
        risk_per_trade: float,
        tp_pips: float,
        pip_size: float,
        symbol_info: dict,
    ):

        self.symbol = symbol
        self.risk_per_trade = risk_per_trade
        self.tp_pips = tp_pips
        self.pip_size = pip_size
        self.info = symbol_info

        # DAILY STATE
        self.today = None
        self.traded_today = False

        # STRATEGY STATE
        self.y_high = None
        self.y_low = None
        self.break_candle = None
        self.direction = None

        self.prev_candle = None

    # ----------------------------------------------------
    # 🔄 DAILY RESET
    # ----------------------------------------------------
    def set_new_day(self, today_date: date, yesterday_high, yesterday_low):
        self.today = today_date
        self.y_high = yesterday_high
        self.y_low = yesterday_low

        self.traded_today = False
        self.break_candle = None
        self.direction = None
        self.prev_candle = None

    # ----------------------------------------------------
    # 🔥 MAIN ENTRY POINT (CALL ON EVERY NEW CANDLE)
    # ----------------------------------------------------
    def on_new_candle(self, candle: Candle, capital: float):
        if self.traded_today:
            self.prev_candle = candle
            return None

        # Need previous candle
        if self.prev_candle is None:
            self.prev_candle = candle
            return None

        # ------------------ C1 BREAK ------------------
        if self.break_candle is None:
            if (
                self.prev_candle.open <= self.y_high
                and self.prev_candle.close > self.y_high
            ):
                self.break_candle = self.prev_candle
                self.direction = "long"

            elif (
                self.prev_candle.open >= self.y_low
                and self.prev_candle.close < self.y_low
            ):
                self.break_candle = self.prev_candle
                self.direction = "short"

            self.prev_candle = candle
            return None

        # ------------------ C2 CONFIRM ------------------
        if candle.timestamp == self.break_candle.timestamp:
            return None

        if self.direction == "long" and candle.close <= self.break_candle.close:
            self._invalidate()
            self.prev_candle = candle
            return None

        if self.direction == "short" and candle.close >= self.break_candle.close:
            self._invalidate()
            self.prev_candle = candle
            return None

        # ------------------ C3 ENTRY ------------------
        trade = self._create_trade(candle, capital)
        self.traded_today = True
        self._invalidate()

        self.prev_candle = candle
        return trade

    # ----------------------------------------------------
    # 🧮 LOT SIZE (REAL & BROKER SAFE)
    # ----------------------------------------------------
    def _calculate_lot(self, capital, entry, sl):
        risk_amount = capital * self.risk_per_trade
        sl_distance = abs(entry - sl)

        if sl_distance <= 0:
            return 0.0

        ticks = sl_distance / self.info["tick_size"]
        loss_per_lot = ticks * self.info["tick_value"]

        raw_lot = risk_amount / loss_per_lot

        step = self.info["lot_step"]
        min_lot = self.info["min_lot"]

        # floor to broker step
        lot = int(raw_lot / step) * step

        # enforce minimum
        if lot < min_lot:
            lot = min_lot

        return round(lot, 3)

    # ----------------------------------------------------
    # 📦 CREATE TRADE OBJECT
    # ----------------------------------------------------
    def _create_trade(self, candle, capital):
        entry = candle.open

        if self.direction == "long":
            sl = self.break_candle.low
            tp = entry + self.tp_pips * self.pip_size
        else:
            sl = self.break_candle.high
            tp = entry - self.tp_pips * self.pip_size

        lot = self._calculate_lot(capital, entry, sl)

        return {
            "symbol": self.symbol,
            "direction": self.direction,
            "lot": lot,
            "entry_price": entry,
            "stop_loss": sl,
            "take_profit": tp,
            "entry_time": candle.timestamp,
        }

    def _invalidate(self):
        self.break_candle = None
        self.direction = None

In [ ]:
# ----------------------------------
# BACKTEST ENGINE
# ----------------------------------
def backtest_last_year(
    csv_path: str,
    output_csv: str,
    initial_capital=10_000,
):
    df = pd.read_csv(csv_path, parse_dates=["timestamp"])
    df = df.sort_values("timestamp")

    capital = initial_capital
    trades = []

    trader = LiveYesterdayHighLowTrader(
        symbol="XAUUSD",
        risk_per_trade=0.03,
        tp_pips=100,
        pip_size=0.1,
        symbol_info={
            "contract_size": 100,
            "tick_size": 0.01,
            "tick_value": 1.0,
            "min_lot": 0.001,
            "lot_step": 0.001,
        },
    )

    current_day = None

    for _, row in df.iterrows():
        candle_day = row["timestamp"].date()

        # -------- NEW DAY --------
        if candle_day != current_day:
            current_day = candle_day

            trader.set_new_day(
                today_date=current_day,
                yesterday_high=row["prev_day_high"],
                yesterday_low=row["prev_day_low"],
            )

        if not row["can_trade"]:
            continue

        candle = Candle(
            timestamp=row["timestamp"],
            open=row["open"],
            high=row["high"],
            low=row["low"],
            close=row["close"],
        )

        trade = trader.on_new_candle(candle, capital)

        if trade:
            # simulate immediate TP/SL in future candles
            exit_price, exit_time, result = simulate_exit(trade, df, row["timestamp"])

            pnl = calculate_pnl(trade, exit_price)

            capital += pnl

            trade.update(
                {
                    "exit_price": exit_price,
                    "exit_time": exit_time,
                    "result": result,
                    "pnl": pnl,
                    "equity": capital,
                }
            )

            trades.append(trade)

    trades_df = pd.DataFrame(trades)
    trades_df.to_csv(output_csv, index=False)

    print(f"✅ Trade book saved to: {output_csv}")
    print(f"Final capital: {round(capital, 3)}")


# ----------------------------------
# EXIT SIMULATION (CANDLE BY CANDLE)
# ----------------------------------
def simulate_exit(trade, df, entry_time):
    future = df[df["timestamp"] > entry_time]

    for _, row in future.iterrows():
        if trade["direction"] == "long":
            if row["low"] <= trade["stop_loss"]:
                return trade["stop_loss"], row["timestamp"], "loss"
            if row["high"] >= trade["take_profit"]:
                return trade["take_profit"], row["timestamp"], "win"
        else:
            if row["high"] >= trade["stop_loss"]:
                return trade["stop_loss"], row["timestamp"], "loss"
            if row["low"] <= trade["take_profit"]:
                return trade["take_profit"], row["timestamp"], "win"

    return None, None, "open"


# ----------------------------------
# PnL CALCULATION (BROKER REAL)
# ----------------------------------
def calculate_pnl(trade, exit_price):
    if exit_price is None:
        return 0.0

    price_move = (
        exit_price - trade["entry_price"]
        if trade["direction"] == "long"
        else trade["entry_price"] - exit_price
    )

    return price_move / 0.01 * trade["lot"] * 1.0


def generate_report(trades: pd.DataFrame, initial_capital: float):
    df = trades.copy()

    # Remove open trades
    df = df[df["result"].isin(["win", "loss"])]

    if df.empty:
        return {"message": "No closed trades to analyze"}

    wins = df[df["result"] == "win"]
    losses = df[df["result"] == "loss"]

    net_pnl = df["pnl"].sum()
    final_capital = initial_capital + net_pnl

    equity = df["equity"]

    report = {
        "initial_capital": round(initial_capital, 3),
        "final_capital": round(final_capital, 3),
        "net_pnl": round(net_pnl, 3),
        "return_%": round(net_pnl / initial_capital * 100, 3),
        "total_trades": len(df),
        "wins": len(wins),
        "losses": len(losses),
        "win_rate_%": round(len(wins) / len(df) * 100, 3),
        "avg_win": round(wins["pnl"].mean(), 3) if not wins.empty else 0,
        "avg_loss": round(losses["pnl"].mean(), 3) if not losses.empty else 0,
        "expectancy": round(df["pnl"].mean(), 3),
        "profit_factor": (
            round(wins["pnl"].sum() / abs(losses["pnl"].sum()), 3)
            if not losses.empty
            else float("inf")
        ),
        "max_drawdown_%": round(_max_drawdown_pct(equity), 3),
    }

    return report


def _max_drawdown_pct(equity):
    peak = equity.cummax()
    drawdown = (equity - peak) / peak
    return drawdown.min() * 100

In [19]:
if __name__ == "__main__":
    # -----------------------------
    # CONFIG
    # -----------------------------
    DATA_CSV = "XAUUSD_M15_with_PDH_PDL.csv"  # your downloaded data
    TRADE_BOOK_CSV = "trade_book_M15_2025.csv"
    INITIAL_CAPITAL = 100

    # -----------------------------
    # RUN BACKTEST
    # -----------------------------
    backtest_last_year(
        csv_path=DATA_CSV,
        output_csv=TRADE_BOOK_CSV,
        initial_capital=INITIAL_CAPITAL,
    )

    # -----------------------------
    # LOAD TRADE BOOK
    # -----------------------------
    trades_df = pd.read_csv(TRADE_BOOK_CSV, parse_dates=["entry_time", "exit_time"])

    # -----------------------------
    # GENERATE REPORT
    # -----------------------------
    report = generate_report(
        trades=trades_df,
        initial_capital=INITIAL_CAPITAL,
    )

    # -----------------------------
    # PRINT REPORT
    # -----------------------------
    print("\n📊 PERFORMANCE SUMMARY")
    print("-" * 40)
    for k, v in report.items():
        print(f"{k:20}: {v}")

    # -----------------------------
    # SAVE REPORT
    # -----------------------------
    report_df = pd.DataFrame([report])
    report_df.to_csv("summary_M15_2025.csv", index=False)

    print("\n✅ Backtest complete")
    print("📁 Trade book  :", TRADE_BOOK_CSV)
    print("📁 Summary    : summary_2025.csv")

✅ Trade book saved to: trade_book_M15_2025.csv
Final capital: 24154.282

📊 PERFORMANCE SUMMARY
----------------------------------------
initial_capital     : 100
final_capital       : 24154.282
net_pnl             : 24054.282
return_%            : 24054.282
total_trades        : 182
wins                : 110
losses              : 72
win_rate_%          : 60.44
avg_win             : 283.791
avg_loss            : -99.482
expectancy          : 132.166
profit_factor       : 4.358
max_drawdown_%      : -13.048

✅ Backtest complete
📁 Trade book  : trade_book_M15_2025.csv
📁 Summary    : summary_2025.csv
